In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='4,5,6,7'

In [ ]:
%run -p ../train_cnf_disentangle_rl.py --data cifar10 --dims 64,64,64 --strides 1,1,1,1 --num_blocks 2 --layer_type concat --multiscale True --rademacher True --batch_size 900 --test_batch_size 500 --save ../experiments_published/cnf_conditional_disentangle_cifar10_bs900_sratio_0_5_drop_0_5_rl_stdscale_15_run3 --resume ../experiments_published/cnf_conditional_disentangle_cifar10_bs900_sratio_0_5_drop_0_5_rl_stdscale_15_run3/epoch_150_checkpt.pth --seed 3 --lr 0.0001 --conditional True --controlled_tol False --train_mode semisup --log_freq 10 --weight_y 0.5 --condition_ratio 0.5 --dropout_rate 0.5 --scale_fac 1.0 --scale_std 15.0
#

/tancode/repos/tan-ffjord/train_cnf_disentangle_rl.py
import argparse
import os
import time
import numpy as np

import torch
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as tforms
from torchvision.utils import save_image

import torch.utils.data as data
from torch.utils.data import Dataset

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300

from PIL import Image
import os.path
import errno
import codecs

import lib.layers as layers
import lib.utils as utils
import lib.multiscale_parallel as multiscale_parallel
import lib.modules as modules
import lib.thops as thops

from train_misc import standard_normal_logprob
from train_misc import set_cnf_options, count_nfe, count_parameters, count_total_time, count_nfe_gate
from train_misc import add_spectral_norm, spectral_norm_power_iteration
from train_misc import create_regularization_fns, get_regularization, append_regularization_to_log

from tensorboardX import SummaryWriter

# 

Files already downloaded and verified
Files already downloaded and verified


ODENVP(
  (transforms): ModuleList(
    (0): StackedCNFLayers(
      (chain): ModuleList(
        (0): LogitTransform()
        (1): CNF_Gate(
          (gate_net): FeedforwardGateI(
            (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (avg_layer): AdaptiveAvgPool2d(output_size=(1, 1))
            (linear_layer): Conv2d(10, 1, kernel_size=(1, 1), stride=(1, 1))
            (sigmoid): Sigmoid()
          )
          (odefunc): RegularizedODEfunc(
            (odefunc): ODEfu